# Imports

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import pandas
import numpy
import data_massage
import itertools
import matplotlib.pyplot as plt

In [ ]:
metadata, full_data, induction_only = data_massage.load_data()
summarized_data = data_massage.summarize_data(metadata, induction_only)

mlp_data = induction_only
features = sorted(["R_mean", "Temp.", "Humidity"])

hidden_layers = tuple(map(lambda p: (len(features) ** p), [3,4,5,4,3]))
feature_permutations = list(itertools.chain.from_iterable(map(lambda r: itertools.combinations(features, r), range(1, 1 + len(features)))))

In [ ]:
results = {}

for perm in feature_permutations:
    feature_set = list(perm)
    mlp_train_x, mlp_test_x, mlp_train_y, mlp_test_y = train_test_split(
          mlp_data[feature_set]
        , mlp_data["label"]
        , test_size=0.2
        , stratify=mlp_data["label"]
        , random_state=7
    )

    label_encoder = LabelEncoder()
    label_encoder.fit(mlp_train_y)
    mlp_train_y = label_encoder.transform(mlp_train_y)
    mlp_test_y = label_encoder.transform(mlp_test_y)
    #MLPClassifier(hidden_layer_sizes=(100,),'activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
    mlp_model = MLPClassifier(hidden_layer_sizes=hidden_layers, activation='tanh', max_iter=10000, alpha=0.01, learning_rate='adaptive')
    mlp_model.fit(mlp_train_x, mlp_train_y)
    prediction = mlp_model.predict(mlp_test_x)
    
    key = ','.join(feature_set)
    results[key] = {"prediction": prediction, "report": classification_report(mlp_test_y, prediction, zero_division=False)}
    
    print(key)
    print(results[key]["report"])
    print("\n\n")